In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
path='/content/drive/MyDrive/2013641_miniproject(data anaytics- article recommendation system)/shared_articles.csv'

In [ ]:
#Retrieving the data from.CSV
articles_df = pd.read_csv(path)
articles_df.describe()

,timestamp,contentId,authorPersonId,authorSessionId
count,3.122000e+03,3.122000e+03,3.122000e+03,3.122000e+03
mean,1.468804e+09,3.284110e+16,3.984819e+17,1.976675e+17
std,7.571512e+06,5.383734e+18,4.387115e+18,5.403753e+18
min,1.459193e+09,-9.222795e+18,-9.120686e+18,-9.212055e+18
25%,1.462369e+09,-4.660869e+18,-1.578288e+18,-4.651280e+18
50%,1.467136e+09,4.436584e+16,-7.092877e+17,4.154103e+17
75%,1.473873e+09,4.745617e+18,3.609194e+18,4.841644e+18
max,1.488308e+09,9.222265e+18,9.210531e+18,9.221043e+18


In [ ]:
print(articles_df.shape) # NUmber of columns and rows

(3122, 13)


In [ ]:
print(articles_df['timestamp'].head(5)) # timestamp is time when an event occurred

0    1459192779
1    1459193988
2    1459194146
3    1459194474
4    1459194497
Name: timestamp, dtype: int64


In [ ]:
print(articles_df['eventType'].value_counts()) # Atricle shared or removed at given timestamp

CONTENT SHARED     3047
CONTENT REMOVED      75
Name: eventType, dtype: int64


In [ ]:
# Filtering and storing articles which is shared 
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
print(articles_df.shape)

(3047, 13)


In [ ]:
print(articles_df['contentId'].head(5)) # Article ID in numeric format

1   -4110354420726924665
2   -7292285110016212249
3   -6151852268067518688
4    2448026894306402386
5   -2826566343807132236
Name: contentId, dtype: int64


In [ ]:
print(articles_df['authorPersonId'].head(5)) # Author Id.

1    4340306774493623681
2    4340306774493623681
3    3891637997717104548
4    4340306774493623681
5    4340306774493623681
Name: authorPersonId, dtype: int64


In [ ]:
#Retrieve unique values
print(len(articles_df['authorPersonId'].unique()))

252


In [ ]:
# Session ID of the author. The author might have created the articles in different sessions
print(articles_df['authorSessionId'].head(5))

1    8940341205206233829
2    8940341205206233829
3   -1457532940883382585
4    8940341205206233829
5    8940341205206233829
Name: authorSessionId, dtype: int64


In [ ]:
# The browser author used.
print(articles_df['authorUserAgent'].tail(5))

3117    Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...
3118    Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...
3119    Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...
3120    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
3121    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
Name: authorUserAgent, dtype: object


In [ ]:
#Retrieve unique browsers
print(len(articles_df['authorUserAgent'].unique()))

115


In [ ]:
# states/regions of the author.
print(articles_df['authorRegion'].tail(5))
print(len(articles_df['authorRegion'].unique()))
print(articles_df['authorRegion'].isnull().sum(axis=0))

3117    SP
3118    GA
3119    SP
3120    MG
3121    SP
Name: authorRegion, dtype: object
20
2378


In [ ]:
#  Country of the author of the articles.
print(articles_df['authorCountry'].tail(5))
print(articles_df['authorCountry'].unique())
print(articles_df['authorCountry'].isnull().sum(axis=0))

3117    BR
3118    US
3119    BR
3120    BR
3121    BR
Name: authorCountry, dtype: object
[nan 'BR' 'CA' 'US' 'AU' 'PT']
2378


In [ ]:
# The formats articles are shared.
print(articles_df['contentType'].unique())
print(articles_df['contentType'].isnull().sum(axis=0))

['HTML' 'RICH' 'VIDEO']
0


In [ ]:
# URL of the articles
print(articles_df['url'].head(5))
print(articles_df['url'].isnull().sum(axis=0))

1    http://www.nytimes.com/2016/03/28/business/dea...
2    http://cointelegraph.com/news/bitcoin-future-w...
3    https://cloudplatform.googleblog.com/2016/03/G...
4    https://bitcoinmagazine.com/articles/ibm-wants...
5    http://www.coindesk.com/ieee-blockchain-oxford...
Name: url, dtype: object
0


In [ ]:
# Title/headline of the articles
print(articles_df['title'].head(5))
print(articles_df['title'].isnull().sum(axis=0))

1    Ethereum, a Virtual Currency, Enables Transact...
2    Bitcoin Future: When GBPcoin of Branson Wins O...
3                         Google Data Center 360° Tour
4    IBM Wants to "Evolve the Internet" With Blockc...
5    IEEE to Talk Blockchain at Cloud Computing Oxf...
Name: title, dtype: object
0


In [ ]:
# Content of the articles.
print(articles_df['text'].head(5))
print(articles_df['text'].isnull().sum(axis=0))

1    All of this work is still very early. The firs...
2    The alarm clock wakes me at 8:00 with stream o...
3    We're excited to share the Google Data Center ...
4    The Aite Group projects the blockchain market ...
5    One of the largest and oldest organizations fo...
Name: text, dtype: object
0


In [ ]:
# language in which the article is written.
print(articles_df['lang'].unique())
print(articles_df['lang'].isnull().sum(axis=0))

['en' 'pt' 'es' 'la' 'ja']
0


In [ ]:
#  restrict the language to English.
articles_df = articles_df[articles_df['lang'] == 'en']
print(articles_df.shape)

(2211, 13)


In [ ]:
# create a dataframe using the pandas library.
articles_df = pd.DataFrame(articles_df, columns=['contentId', 'authorPersonId', 'content', 'title', 'text'])

In [ ]:
# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(articles_df['text'])

In [ ]:
# Output the shape of tfidf_matrix
print(tfidf_matrix.shape)
# print(tfidf.get_stop_words().pop())

# Array mapping from feature integer indices to feature name.
print(tfidf.get_feature_names()[5000:5010])

(2211, 45496)
['banter', 'baptista', 'baptiste', 'baptized', 'bar', 'barack', 'barani', 'baratheon', 'barauskas', 'barb']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# use the cosine_similarity() function of the sklearn library.
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix, True)
display(cosine_sim.shape)
display(cosine_sim)

(2211, 2211)

array([[1.        , 0.02842053, 0.01414884, ..., 0.04717028, 0.08436331,
        0.01859574],
       [0.02842053, 1.        , 0.02096081, ..., 0.0256655 , 0.03187741,
        0.01120014],
       [0.01414884, 0.02096081, 1.        , ..., 0.02092281, 0.04240744,
        0.        ],
       ...,
       [0.04717028, 0.0256655 , 0.02092281, ..., 1.        , 0.05457163,
        0.07645089],
       [0.08436331, 0.03187741, 0.04240744, ..., 0.05457163, 1.        ,
        0.03965418],
       [0.01859574, 0.01120014, 0.        , ..., 0.07645089, 0.03965418,
        1.        ]])

In [ ]:
# Reset index of main DataFrame and construct reverse mapping as before
metadata = articles_df.reset_index()
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()
display(indices[:10])

title
Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's                                          0
Bitcoin Future: When GBPcoin of Branson Wins Over USDcoin of Trump                                               1
Google Data Center 360° Tour                                                                                     2
IBM Wants to "Evolve the Internet" With Blockchain Technology                                                    3
IEEE to Talk Blockchain at Cloud Computing Oxford-Con - CoinDesk                                                 4
Banks Need To Collaborate With Bitcoin and Fintech Developers                                                    5
Blockchain Technology Could Put Bank Auditors Out of Work                                                        6
Why Decentralized Conglomerates Will Scale Better than Bitcoin - Interview with OpenLedger CEO - Bitcoin News    7
The Rise And Growth of Ethereum Gets Mainstream Coverage                  

In [ ]:
# Function that takes in article title as input and outputs most similar articles

# it finds the index of the input title. Since we have to remove it later, it
# iterates over the length of the cosine similarity matrix and checks for the 
# distance of the articles from the input article we passed.


def get_recommendations(title, indices, cosine_sim, data):
    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the article indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar articles
    return data['title'].iloc[movie_indices]

In [ ]:
# Printing the given recomendaions
print(get_recommendations('Blockchain Technology Could Put Bank Auditors Out of Work', indices, cosine_sim
                            , metadata))

727                      The Blockchain is the new Google
730      Confused by blockchains? Revolution vs Evolution
1671    Banks find blockchain hard to put into practic...
1701    Are you a bitcoin denialist? - Chris Skinner's...
766     Smart Managers Don't Compare People to the "Av...
2140    4 Blockchain Macro Trends: Where to Place Your...
1758    Blockchain Will Be Used By 15% of Big Banks By...
17      From fine wine to lotteries: Blockchain tech t...
307          Blockchain won't kill banks: Bitcoin pioneer
154     Venture Capitalists Inject Nearly Half a Billi...
Name: title, dtype: object
